In [1]:
import networkx as nx
import scipy as sc
import matplotlib.pyplot as plt
from itertools import combinations
import operator
from gurobipy import *
import random as rd

In [2]:
commodities = ['first','second','third','fourth']

In [3]:
G = nx.Graph(nx.barabasi_albert_graph(1000, 2))
pos=nx.spring_layout(G)

# G=nx.DiGraph()
# filename="./roadNet-TX.txt"
# f=open(filename)
# for row in f:
#     myrow=row.split()
#     G.add_edge(myrow[0], myrow[1])

In [4]:
for (i,j,d) in G.edges(data=True):
    d['weight']=rd.randint(800,2000)

In [5]:
for k in commodities:
    for (i,d) in G.nodes(data=True):
        d[k]=0
        


In [6]:
for (i,j,d) in G.edges(data=True):
    d['capacity']=1

In [9]:
supplier=rd.sample(G.nodes(),4)
terminals=rd.sample(G.nodes(),4)

/var/folders/np/xyxhbcvd7ks47j34qd3svpf00000gn/T/ipykernel_61850/3957267518.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  supplier=rd.sample(G.nodes(),4)
/var/folders/np/xyxhbcvd7ks47j34qd3svpf00000gn/T/ipykernel_61850/3957267518.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  terminals=rd.sample(G.nodes(),4)


In [10]:
for i,d in G.nodes(data=True):
    if i==supplier[0]:
        d['first']=-2
        print(i,d['first'])
    if i==supplier[1]:
        d['second']=-2
        print(i,d['second'])
    if i==supplier[2]:
        d['third']=-2
    if i==supplier[3]:
        d['third']=-2
    

29 -2
212 -2


In [11]:
for i,d in G.nodes(data=True):
    if i==terminals[0]:
        d['first']=2
        print(i,d['first'])
    if i==terminals[1]:
        d['second']=2
        print(i,d['second']) 
    if i==terminals[2]:
        d['third']=2
    if i==terminals[3]:
        d['third']=2

13 2
433 2


In [12]:
from gurobipy import *
m=Model('commodityflow')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-20


In [13]:
x = {}
for k in commodities:
    for i,d in G.nodes(data=True):
        for j in G.nodes():
            if (i,j) in G.edges():
                x[(k,i,j)] = m.addVar(lb=0,ub=1,vtype=GRB.BINARY,name='x.{0}.{1}.{2}'.format(k, i, j))

In [14]:
for (i,j,d) in G.edges(data=True):
    sum9=0
    for k in commodities:
        sum9+=x[(k,i,j)]
    m.addConstr(sum9<=d['capacity'])

In [15]:
for k in commodities:
    for i,d in G.nodes(data=True):
        sum1=0
        sum2=0
        for j in G.nodes():
            if i!=j:
                if (i,j) in G.edges():
                    sum1+=x[(k,i,j)]
                if (j,i) in G.edges():
                    sum2+=x[(k,j,i)]
        m.addConstr((sum2-sum1)==d[k])

In [16]:
obj=0
for k in commodities:
    for i,j,d in G.edges(data=True):
        obj+=x[(k,i,j)]*d['weight']

In [17]:
m.update()
m.setObjective(obj,GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5996 rows, 15968 columns and 39920 nonzeros
Model fingerprint: 0x734e43b6
Variable types: 0 continuous, 15968 integer (15968 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 21 rows and 59 columns
Presolve time: 0.05s
Presolved: 5975 rows, 15909 columns, 39772 nonzeros
Variable types: 0 continuous, 15909 integer (15909 binary)

Root relaxation: infeasible, 4122 iterations, 0.16 seconds (0.22 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 1 nodes (4122 simplex iterations) in 0.24 seconds (0.28 w